In [ ]:
%load_ext autoreload
%autoreload 2

# MLflow Classification Recipe Notebook

This notebook runs the MLflow Classification Recipe on Databricks and inspects its results. For more information about the MLflow Classification Recipe, including usage examples, see the [Classification Recipe overview documentation](https://mlflow.org/docs/latest/recipes.html#classification-recipe) the [Classification Recipe API documentation](https://mlflow.org/docs/latest/python_api/mlflow.recipes.html#module-mlflow.recipes.classification.v1.recipe).

In [ ]:
from mlflow.recipes import Recipe

r = Recipe(profile="local")


In [ ]:
r.clean()

In [ ]:
r.inspect()

In [ ]:
r.run("ingest")

In [ ]:
r.run("split")

In [ ]:
r.run("transform")

In [ ]:
r.run("train")

In [ ]:
r.run("evaluate")

In [ ]:
r.run("register")

In [ ]:
r.inspect("train")

In [ ]:
training_data = r.get_artifact("transformer")
help(training_data.func)